# 範例 11：通用聊天程式

可以在 Ollama 和 LM Studio 之間切換的聊天機器人！

## 學習目標
- 建立支援多種後端的聊天程式
- 學會抽象化 API 差異
- 實作靈活的程式架構

## 為什麼需要通用程式？
- 可以根據需求選擇不同的 AI 後端
- 方便比較不同模型的效果
- 程式碼更容易維護和擴展

## 前置需求
- Ollama 或 LM Studio 運行中
- 安裝 openai 套件：`pip install openai`

## Step 1: 匯入套件

In [ ]:
import requests
from openai import OpenAI

## Step 2: 建立通用聊天機器人類別

In [ ]:
class UniversalChatBot:
    """
    通用聊天機器人
    支援 Ollama 和 LM Studio 兩種後端
    """

    def __init__(self, backend="lmstudio", model=None):
        """
        初始化聊天機器人

        參數：
            backend: "ollama" 或 "lmstudio"
            model: 模型名稱（可選）
        """
        self.backend = backend
        self.messages = []

        if backend == "lmstudio":
            self.client = OpenAI(
                base_url="http://localhost:1234/v1",
                api_key="not-needed"
            )
            self.model = model or "gpt-oss-120b"
        elif backend == "ollama":
            self.url = "http://localhost:11434/api/chat"
            self.model = model or "gpt-oss:120b"
        else:
            raise ValueError("backend 必須是 'ollama' 或 'lmstudio'")
        
        print(f"已初始化 {backend.upper()} 後端，使用模型：{self.model}")

    def chat(self, user_message):
        """發送訊息並獲得回應"""

        self.messages.append({"role": "user", "content": user_message})

        if self.backend == "lmstudio":
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages
            )
            ai_message = response.choices[0].message.content

        else:  # ollama
            data = {
                "model": self.model,
                "messages": self.messages,
                "stream": False
            }
            response = requests.post(self.url, json=data)
            result = response.json()
            ai_message = result["message"]["content"]

        self.messages.append({"role": "assistant", "content": ai_message})
        return ai_message
    
    def set_system_prompt(self, system_prompt):
        """設定系統提示詞"""
        self.messages = [{"role": "system", "content": system_prompt}]
    
    def clear_history(self):
        """清除對話歷史"""
        self.messages = []
        print("對話歷史已清除！")
    
    def get_backend_info(self):
        """取得後端資訊"""
        return {
            "backend": self.backend,
            "model": self.model,
            "message_count": len(self.messages)
        }

## Step 3: 使用 LM Studio 後端

In [ ]:
# 建立使用 LM Studio 的聊天機器人
lm_bot = UniversalChatBot(backend="lmstudio")

print("\n=== LM Studio 測試 ===")
response = lm_bot.chat("你好！請用一句話介紹你自己。")
print(f"AI：{response}")

## Step 4: 使用 Ollama 後端

In [ ]:
# 建立使用 Ollama 的聊天機器人
# 如果沒有運行 Ollama，這個會失敗
try:
    ollama_bot = UniversalChatBot(backend="ollama")
    print("\n=== Ollama 測試 ===")
    response = ollama_bot.chat("你好！請用一句話介紹你自己。")
    print(f"AI：{response}")
except Exception as e:
    print(f"Ollama 連接失敗：{e}")
    print("請確認 Ollama 是否正在運行")

## Step 5: 比較不同後端

In [ ]:
def compare_backends(question):
    """
    比較不同後端對同一問題的回答
    """
    print(f"問題：{question}")
    print("=" * 50)
    
    # LM Studio
    try:
        lm = UniversalChatBot(backend="lmstudio")
        lm_response = lm.chat(question)
        print(f"\n[LM Studio]")
        print(lm_response[:300] + "..." if len(lm_response) > 300 else lm_response)
    except Exception as e:
        print(f"\n[LM Studio] 錯誤：{e}")
    
    # Ollama
    try:
        ollama = UniversalChatBot(backend="ollama")
        ollama_response = ollama.chat(question)
        print(f"\n[Ollama]")
        print(ollama_response[:300] + "..." if len(ollama_response) > 300 else ollama_response)
    except Exception as e:
        print(f"\n[Ollama] 錯誤：{e}")

In [ ]:
# 比較兩個後端
compare_backends("什麼是遞迴？")

## Step 6: 工廠函數設計模式

In [ ]:
def create_chatbot(backend="auto", model=None):
    """
    工廠函數：自動選擇可用的後端
    
    參數：
        backend: "auto", "lmstudio", 或 "ollama"
        model: 模型名稱
    
    回傳：
        UniversalChatBot 實例
    """
    if backend != "auto":
        return UniversalChatBot(backend=backend, model=model)
    
    # 自動檢測可用的後端
    print("正在檢測可用的後端...")
    
    # 先嘗試 LM Studio
    try:
        response = requests.get("http://localhost:1234/v1/models", timeout=2)
        if response.status_code == 200:
            print("找到 LM Studio！")
            return UniversalChatBot(backend="lmstudio", model=model)
    except:
        pass
    
    # 再嘗試 Ollama
    try:
        response = requests.get("http://localhost:11434/api/tags", timeout=2)
        if response.status_code == 200:
            print("找到 Ollama！")
            return UniversalChatBot(backend="ollama", model=model)
    except:
        pass
    
    raise RuntimeError("找不到可用的 AI 後端！請啟動 LM Studio 或 Ollama。")

In [ ]:
# 自動選擇後端
try:
    auto_bot = create_chatbot(backend="auto")
    print(f"\n使用的後端：{auto_bot.get_backend_info()}")
    print(f"\n回答：{auto_bot.chat('1+1等於多少？')}")
except Exception as e:
    print(f"錯誤：{e}")

## 架構設計說明

```
┌─────────────────────────────────────┐
│         UniversalChatBot            │
├─────────────────────────────────────┤
│  - backend: str                     │
│  - messages: list                   │
│  - model: str                       │
├─────────────────────────────────────┤
│  + chat(message) -> str             │
│  + set_system_prompt(prompt)        │
│  + clear_history()                  │
└─────────────────────────────────────┘
              │
    ┌─────────┴─────────┐
    ▼                   ▼
┌─────────┐       ┌──────────┐
│ Ollama  │       │ LM Studio │
│  API    │       │   API    │
└─────────┘       └──────────┘
```

這種設計讓使用者不需要關心底層使用哪個 API！

## 練習

In [ ]:
# 建立你自己的聊天機器人
my_bot = create_chatbot(backend="auto")

# 設定角色
my_bot.set_system_prompt("你是一位友善的助手，用繁體中文回答。")

# 開始對話
print(my_bot.chat("你好！"))

## 重點回顧

1. **統一介面**：使用同一個類別處理不同後端
2. **工廠模式**：使用 `create_chatbot()` 自動選擇後端
3. **錯誤處理**：優雅地處理連接失敗
4. **可擴展性**：容易加入新的後端支援

## 下一步

接下來我們將學習 RAG（檢索增強生成）技術！